# COMP5318 - Machine Learning and Data Mining: Assignment 1

In [1]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support
import graphviz

# TODO
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

%matplotlib inline

['train.csv']


In [2]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 

In [3]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

In [4]:
# Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(
    data_train_feature, data_train_label, random_state=0, stratify=data_train_label, test_size=0.0)

In [33]:
# Performance Metrics Calculator Helper 
def performance(y_true, y_pred, type):
    precision_test = precision_recall_fscore_support(y_true, y_pred, average='macro')
    print("Accuracy on " + type + " set: {:.3f}".format(accuracy_score(y_true, y_pred)))
    print("Precision on " + type + " set: {:.3f}".format(precision_test[0]))
    print("Recall on " + type + " set: {:.3f}".format(precision_test[1]))
    print("F-Score on " + type + " set: {:.3f}".format(precision_test[2]))

## DATA PRE-PROCESSING FOR TRAINING DATA

In [12]:
# Normalisation
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm  = scaler.transform(X_test)

pd.DataFrame(X_train_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,...,0.462745,0.38,0.082353,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22496,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22497,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0
22498,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.0,0.0


In [13]:
# Dimension Reduction
pca = PCA(n_components=0.9).fit(X_train_norm)

X_train_pca = pca.transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)

pd.DataFrame(X_train_pca)

,0,1,2,3,4,...,79,80,81,82,83
0,2.927449,-5.553593,4.019518,-1.395135,1.064630,...,-0.011555,-0.025837,-0.095283,-0.109759,-0.024098
1,1.992421,0.442999,-2.368003,0.811313,-1.177249,...,-0.479448,0.015362,-0.228318,-0.181703,-0.037604
2,2.916436,-4.533643,2.285697,-2.685905,0.292165,...,-0.053102,0.513047,-0.128389,0.073825,-0.079035
3,-6.189816,1.348044,-0.645078,-2.423576,1.313570,...,-0.153498,-0.184887,0.326095,0.042007,-0.248368
4,-2.974141,-4.625361,1.107169,0.583185,0.232016,...,0.265475,0.093921,0.410109,0.028499,0.002914
...,...,...,...,...,...,...,...,...,...,...,...
22495,1.257244,5.409011,5.380687,3.621838,-2.909050,...,-0.069430,0.024253,-0.329398,-0.153290,0.420994
22496,1.754955,-4.031853,0.866494,-0.421730,0.518514,...,-0.398757,-0.208873,0.206416,-0.112603,0.073324
22497,-6.374198,0.561054,-0.780198,-2.838871,1.389735,...,0.245362,0.187905,-0.085312,-0.223798,-0.003016
22498,-5.730576,2.660773,-0.073577,-3.724108,1.928655,...,0.021028,0.012773,0.234765,0.361081,0.209532


In [ ]:
%%time
# Create a KNN Classifier using best parameters 
knn = KNeighborsClassifier(n_neighbors=5, p=1)
knn.fit(X_train_pca, y_train)

y_pred_train = knn.predict(X_train_pca)
performance(y_train, y_pred_train, 'train')

KeyError: 'n_neighbors'

In [35]:
%%time
# Create a SVM Classifier using best parameters 
svm = SVC(C=10, gamma=0.01)
svm.fit(X_train_pca, y_train)

y_pred_train = svm.predict(X_train_pca)
performance(y_train, y_pred_train, 'train')

Accuracy on test set: 0.887
Precision on test set: 0.886
Recall on test set: 0.887
F-Score on test set: 0.886
Accuracy on train set: 0.934
Precision on train set: 0.934
Recall on train set: 0.934
F-Score on train set: 0.933
CPU times: user 56.9 s, sys: 301 ms, total: 57.2 s
Wall time: 57.5 s


## DATA PRE-PROCESSING FOR BLIND TESTING DATA

In [29]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv', index_col=0) 

In [30]:
# Data Normalisation
output_test_norm = scaler.transform(data_test_df.to_numpy())

pd.DataFrame(output_test_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
3,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4996,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4997,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4998,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0


In [31]:
# Dimension Reduction
output_test_pca = pca.transform(output_test_norm)

pd.DataFrame(output_test_pca)

,0,1,2,3,4,...,79,80,81,82,83
0,2.030060,-5.433903,1.980836,-1.898502,-1.143763,...,0.163122,0.084214,0.142833,0.013530,-0.139753
1,1.971218,-5.661186,1.332592,-1.375247,-0.702272,...,-0.407891,0.289796,-0.219731,-0.249055,-0.450584
2,0.527241,-5.844876,1.605396,-1.962709,-2.800743,...,-0.056665,0.077233,0.170281,0.118583,0.053545
3,-0.334794,-3.352547,-0.780979,1.740940,0.259736,...,-0.145004,-0.078867,-0.272437,-0.101939,0.117087
4,-1.495075,-2.585312,-0.883327,0.401559,0.713086,...,0.370981,0.381618,-0.013033,0.024698,0.149657
...,...,...,...,...,...,...,...,...,...,...,...
4995,2.789104,-3.193621,0.806616,0.593713,-0.233942,...,0.041395,-0.117392,-0.476785,-0.203433,-0.099679
4996,2.489568,5.947833,-0.281650,1.209622,3.791362,...,-0.535142,0.092127,0.591728,0.717596,-0.166519
4997,-5.653882,-0.605367,-1.218067,3.388302,-0.931545,...,-0.497402,-0.183685,-0.146962,0.410792,-0.063161
4998,7.083845,1.511506,0.441606,1.107137,0.528356,...,-0.400921,0.021544,-0.081451,0.188384,-0.141796


In [32]:
# Helper function to export csv file storing predictions of a classifier on the blind test set 
def export_predictions(filename, classifier):
    predictions = []
    filepath = './Output/' + filename + '.csv'

    for i in output_test_pca:
        prediction = classifier.predict([list(i)])
        predictions.append(prediction[0])

    output_df = pd.DataFrame(predictions, columns = ['label'])
    output_df.to_csv(filepath, sep=",", float_format='%d', index_label="id")


In [ ]:
export_predictions('knn_kaggle', knn)
export_predictions('svm_kaggle', svm)

Example Usage:

`export_predictions('knn', knn)`. This will create a file "knn.csv" in the Output folder which will store the predictions of the KNN classifier for the blind testing data. 